In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Define dimensions
num_rows = 6
num_cols = 4

# Generate random data
data = np.random.rand(num_rows, num_cols)

# Create row and column labels
row_labels = [f'Row{i+1}' for i in range(num_rows)]
col_labels = [f'Col{j+1}' for j in range(num_cols)]

# Create DataFrame
df = pd.DataFrame(data, index=row_labels, columns=col_labels)

# Display the DataFrame
print("DataFrame:")
print(df)

# Summary (info)
#print("\nSummary (info):")
#print(df.info())

# Shape
print("\nShape of DataFrame:")
print(df.shape)

# Statistics
print("\nDescriptive Statistics:")
print(df.describe())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Load the Iris dataset
iris = sns.load_dataset('iris')

# Display the first few rows (optional)
print("First few rows of the Iris dataset:")
print(iris.head())

# List of numerical columns
numerical_cols = iris.select_dtypes(include=['float']).columns

# Create boxplots for each numerical column
plt.figure(figsize=(12, 8))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 2, i)
    plt.boxplot(iris[col])
    plt.title(f'Boxplot of {col}')
    plt.ylabel(col)

# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import pandas as pd
import plotly.express as px

# Load the Tips dataset
tips = sns.load_dataset('tips')

# Group the data by sex, day, and time, then sum the tips
grouped = tips.groupby(['sex', 'day', 'time'])['tip'].sum().reset_index()

# Create the sunburst chart
fig = px.sunburst(
    grouped,
    path=['sex', 'day', 'time'],  # hierarchy: sex → day → time
    values='tip',
    title='Total Tips by Gender, Day, and Time',
    color='tip',
    color_continuous_scale='Blues'
)

# Show the plot
fig.show()

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Load the Tips dataset
tips = sns.load_dataset('tips')

# Define the feature and target variable
X = tips[['total_bill']]  # Feature
y = tips['tip']           # Target

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X, y)

# Display model parameters
print(f"Intercept: {model.intercept_:.2f}")
print(f"Coefficient: {model.coef_[0]:.2f}")

# Predict tip for a total bill of $30
sample_bill = np.array([[30]])
predicted_tip = model.predict(sample_bill)
print(f"Predicted tip for a $30 bill: ${predicted_tip[0]:.2f}")